In [2]:
import pandas as pd 
import numpy as np

In [3]:
rename_dict = {
    'ISBN': 'isbn',
    'Book-Author': 'authors',
    'Book-Title': 'title',
    'Year-Of-Publication': 'original_publication_year',
    'Image-URL-S': 'small_image_url'
}

ratings_l = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\larger_set\ratings.csv", sep = ';', encoding = 'latin-1')
users_l = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\larger_set\users.csv", sep = ';', encoding = 'latin-1')
books_l = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\larger_set\books.csv", sep = ';', encoding = 'latin-1', error_bad_lines=False)
books_l = books_l.rename(columns = rename_dict).drop(['Publisher', 'Image-URL-M','Image-URL-L'], axis = 1)
books_l['original_publication_year'] =  pd.to_numeric(books_l['original_publication_year'], errors='coerce', downcast='integer')

# Import smaller set
ratings_s = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\goodbooks-10k\ratings_10k.csv")
books_s = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\goodbooks-10k\books_10k.csv")
books_s_redux = books_s[['isbn','authors','title','original_publication_year','small_image_url']]
books_s_redux['original_publication_year'] = pd.to_numeric(books_s['original_publication_year'], errors='coerce', downcast='integer')

books_df = pd.concat([books_l, books_s_redux], axis = 0)
books_df = books_df.drop_duplicates(subset = 'isbn')

C:\Users\penna\AppData\Local\Temp\ipykernel_1932\2935986721.py:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books_l = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\larger_set\books.csv", sep = ';', encoding = 'latin-1', error_bad_lines=False)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expe

In [4]:
rating_column_dict = {
    'User-ID': 'user_id_adj',
    'ISBN': 'isbn',
    'Book-Rating': 'rating_adj'
}

ratings_s['isbn'] = ratings_s['book_id'].map(books_s[['id', 'isbn']].set_index('id').to_dict()['isbn'])
ratings_s['rating_adj'] = ratings_s['rating'] * 2
ratings_s['user_id_adj'] = ratings_s['user_id'] + ratings_l['User-ID'].max() + 1
ratings_s_redux = ratings_s[['user_id_adj', 'isbn', 'rating_adj']]
ratings_l = ratings_l.rename(columns = rating_column_dict)
ratings_df = pd.concat([ratings_l, ratings_s_redux], axis = 0)



In [8]:
len(ratings_df['user_id_adj'].unique())

158707